## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-26-01-01-08 +0000,nyt,"With Trump’s Asia Trip, Speculation Mounts of ...",https://www.nytimes.com/2025/10/25/world/asia/...
1,2025-10-26-01-00-00 +0000,wsj,Milei’s Free-Market Experiment Hangs in the Ba...,https://www.wsj.com/world/americas/mileis-free...
2,2025-10-26-01-00-00 +0000,wsj,A Hiking Death Roils a Fashion Empire and the ...,https://www.wsj.com/business/a-hiking-death-ro...
3,2025-10-26-01-00-00 +0000,wsj,"“You have zero zoning conditions, you’re the p...",https://www.wsj.com/politics/policy/trump-whit...
4,2025-10-26-00-54-32 +0000,bbc,A humming annoyance or jobs boom? Life next to...,https://www.bbc.com/news/articles/c93dnnxewdvo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2454/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,40
249,he,14
1,asia,11
83,new,10
66,war,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
157,2025-10-25-09-38-00 +0000,wsj,Trump Pitches Meeting With Kim Jong Un During ...,https://www.wsj.com/world/asia/trump-pitches-m...,92
111,2025-10-25-14-56-21 +0000,nypost,Trump ‘open’ to meeting with Kim Jong Un durin...,https://nypost.com/2025/10/25/world-news/trump...,91
160,2025-10-25-09-10-25 +0000,bbc,Trump says he wants to meet Kim Jong Un on Asi...,https://www.bbc.com/news/articles/cn09gg8v721o...,82
34,2025-10-25-22-25-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...,82
72,2025-10-25-19-01-04 +0000,nypost,Trump hosts Emir of Qatar on Air Force One as ...,https://nypost.com/2025/10/25/us-news/trump-ho...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
157,92,2025-10-25-09-38-00 +0000,wsj,Trump Pitches Meeting With Kim Jong Un During ...,https://www.wsj.com/world/asia/trump-pitches-m...
19,48,2025-10-25-23-10-24 +0000,nypost,NYC voters run to the polls on first day of ea...,https://nypost.com/2025/10/25/us-news/nyc-vote...
34,42,2025-10-25-22-25-00 +0000,wsj,Trump Says He Will Raise Tariffs on Canada by ...,https://www.wsj.com/politics/policy/trump-hike...
3,36,2025-10-26-01-00-00 +0000,wsj,"“You have zero zoning conditions, you’re the p...",https://www.wsj.com/politics/policy/trump-whit...
110,36,2025-10-25-15-21-49 +0000,nypost,Chilling footage shows Russian drone hunting d...,https://nypost.com/2025/10/25/world-news/chill...
9,27,2025-10-26-00-07-12 +0000,latimes,Exxon sues California over new laws requiring ...,https://www.latimes.com/california/story/2025-...
55,25,2025-10-25-20-55-20 +0000,nypost,Rubio vows to return bodies of all dead hostag...,https://nypost.com/2025/10/25/world-news/rubio...
40,25,2025-10-25-21-58-46 +0000,nypost,Louvre heist being investigated as inside job ...,https://nypost.com/2025/10/25/world-news/louvr...
12,25,2025-10-25-23-41-30 +0000,nypost,Mikie Sherrill isn’t the first politician to f...,https://nypost.com/2025/10/25/business/mikie-s...
58,25,2025-10-25-20-49-15 +0000,nypost,Here are the NYC neighborhoods where most sex ...,https://nypost.com/2025/10/25/us-news/nyc-home...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
